In [1]:
import json
import os
import requests
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
TAGS_FILE = "testing/test.json"
INPUT_FILE = "../resources/videos-scrap.json"
OUTPUT_FILE = "sources/videos_with_tags.json"
API_URL = "https://models.github.ai/inference/chat/completions"
API_KEY = os.getenv("VITE_OPENAI_API_KEY")
MODEL = "openai/gpt-4.1-nano"

In [4]:
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
tag_store = {}

In [6]:
new_tags = []
for item in data:
    for video in item["videos"]:
        if "tag" in video:
            if video['tag'] not in new_tags:
                new_tags.append(video['tag'])

In [7]:
len(new_tags)

68

In [8]:
import time

MAX_RETRIES = 5
WAIT_SECONDS = 5 

In [58]:
with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
    data_helper = json.load(f)

In [59]:
videos_no_tag = []
for item in data_helper:
    for video in item["videos"]:
        if not video.get("tag"):
            videos_no_tag.append(video)

In [60]:
len(videos_no_tag)

3

In [52]:
tag_store

{'Cap-26: Alclad 2, Pinturas Metálicas Especializadas En Modelismo': 'Tutorial',
 'Unboxing X Speed Build Liger Zero Panzer by. BT Model': 'Unboxing',
 'Girls in love- Andreas Dorau (Subtitulos en español)': 'Pop'}

In [53]:
for video in videos_no_tag:
    print(f"Processing video: {video['titulo']}")  

Processing video: 📌serigrafia con TINTA PUFF de alto relieve🔥parte 2 #viral #estampados #serigrafiatextil #estampado
Processing video: Aplicando Técnica de puff alto relieve con plastisol #puf #plastisol #serigrafia
Processing video: Todas las legiones de Marines espaciales explicadas! | Warhammer 40k historia (Parte 1/3)
Processing video: Introducción a Warhammer 40.000
Processing video: Cómo hacer una base de diorama oceánica realista | Construyendo el AFV Club U-boot Tipo VII D
Processing video: Starboy x I Feel It Coming
Processing video: Sunflower, Post Malone, Swae Lee, (Extended Versión)
Processing video: El Cadav3r de la Novia : La Historia en 1 Video
Processing video: EL DIA QUE LA TIERRA EXPLOTO | RESUMEN EN 22 MINUTOS
Processing video: LAS INJUSTICIAS A UN PROFESOR : PELICULA PERUANA
Processing video: América Noticias: Mujer ebria causó accidente de tránsito y protagonizó escándalo en la vía pública
Processing video: Mitski - My Love Mine All Mine (Extended Version)
Processi

In [54]:
titulos = "\n".join(v["titulo"] for v in videos_no_tag)
util_tags = ",".join(new_tags)   

prompt = f"Asigna una etiqueta (tag) temática de una sola palabra para cada uno de los siguientes títulos de video:\n\n{titulos}"

if util_tags:
    prompt += f"\n\nPuedes utiliza las siguientes etiquetas o agregar otras nuevas: {util_tags}"
prompt += "\n\nFormato de respuesta:\n- [Título]=># [Tag]"

print(f"📝 Generando tags para {len(videos_no_tag)} videos...")    
print(f"Prompt: {prompt}")

📝 Generando tags para 40 videos...
Prompt: Asigna una etiqueta (tag) temática de una sola palabra para cada uno de los siguientes títulos de video:

📌serigrafia con TINTA PUFF de alto relieve🔥parte 2 #viral #estampados #serigrafiatextil #estampado
Aplicando Técnica de puff alto relieve con plastisol #puf #plastisol #serigrafia
Todas las legiones de Marines espaciales explicadas! | Warhammer 40k historia (Parte 1/3)
Introducción a Warhammer 40.000
Cómo hacer una base de diorama oceánica realista | Construyendo el AFV Club U-boot Tipo VII D
Starboy x I Feel It Coming
Sunflower, Post Malone, Swae Lee, (Extended Versión)
El Cadav3r de la Novia : La Historia en 1 Video
EL DIA QUE LA TIERRA EXPLOTO | RESUMEN EN 22 MINUTOS
LAS INJUSTICIAS A UN PROFESOR : PELICULA PERUANA
América Noticias: Mujer ebria causó accidente de tránsito y protagonizó escándalo en la vía pública
Mitski - My Love Mine All Mine (Extended Version)
Jaze, Elsa y Elmar - un mensaje de voz (feat. Elsa y Elmar) (Video Lyric)
F

In [14]:
new_tags

['Urbano',
 'Debate',
 'Alternativo',
 'Electrónico',
 'Rock',
 'Electro',
 'Lounge',
 'Anime',
 'Pop',
 'Indie',
 'Alternative',
 'Informativo',
 'Comedia',
 'Reflexión',
 'Tutorial',
 'Relajación',
 'Animación',
 'Satírico',
 'Verano',
 'Ciencia',
 'Tecnología',
 'Drama',
 'Sociedad',
 'Motivación',
 'Social',
 'Educación',
 'K-Pop',
 'Latin',
 'Aventura',
 'CienciaFicción',
 'Música',
 'Historia',
 'Cultura',
 'Parodia',
 'Salud',
 'Religión',
 'Hobby',
 'Entretenimiento',
 'Electró',
 'Videojuegos',
 'Acción',
 'Juegos',
 'Rap',
 'Hip-Hop',
 'Unboxing',
 'Viajes',
 'Cine',
 'Jazz',
 'Clásica',
 'Debates',
 'Humor',
 'Dramas',
 'Consejos',
 'Vida',
 'RegionalMexicano',
 'Motociclismo',
 'Dance',
 'Comics',
 'Terror',
 'Fantasía',
 'J-Pop',
 'Documental',
 'Creatividad',
 'Sociedaddebate',
 'Tradicional',
 'Instrumental',
 'Soul',
 'Infomativo']

In [19]:
with open('sources/videos_with_tags.json', 'r', encoding='utf-8') as f:
    data_2 = json.load(f)

In [55]:
def get_tags_from_response(response):
    lines = response.split("\n")
    tags = {}
    for line in lines:
        if "=># " in line:
            try:
                title, tag = line.strip("- ").split("=># ", 1)
                tags[title.strip()] = tag.strip()
                print(f"Title: {title}, Tag: {tag}")
            except ValueError:
                print(f"❌ Línea con formato inválido: {line}")
    return tags

# Leer el contenido del archivo como string completo
with open('sources/helper.md', 'r', encoding='utf-8') as f:
    contenido = f.read()

# Llamar la función con el contenido leído
tags_obtenidos = get_tags_from_response(contenido)

Title: 📌serigrafia con TINTA PUFF de alto relieve🔥parte 2 #viral #estampados #serigrafiatextil #estampado, Tag: Tutorial
Title: Aplicando Técnica de puff alto relieve con plastisol #puf #plastisol #serigrafia, Tag: Tutorial
Title: Todas las legiones de Marines espaciales explicadas! | Warhammer 40k historia (Parte 1/3), Tag: Historia
Title: Introducción a Warhammer 40.000, Tag: Cultura
Title: Cómo hacer una base de diorama oceánica realista | Construyendo el AFV Club U-boot Tipo VII D, Tag: Hobby
Title: Starboy x I Feel It Coming, Tag: Pop
Title: Sunflower, Post Malone, Swae Lee, (Extended Versión), Tag: Pop
Title: El Cadav3r de la Novia : La Historia en 1 Video, Tag: Cine
Title: EL DIA QUE LA TIERRA EXPLOTO | RESUMEN EN 22 MINUTOS, Tag: CienciaFicción
Title: LAS INJUSTICIAS A UN PROFESOR : PELICULA PERUANA, Tag: Drama
Title: América Noticias: Mujer ebria causó accidente de tránsito y protagonizó escándalo en la vía pública, Tag: Informativo
Title: Mitski - My Love Mine All Mine (Exten

In [56]:
for item in data_helper:
    for video in item["videos"]:
        if video["titulo"] in tags_obtenidos:
            video["tag"] = tags_obtenidos[video["titulo"]]
            print(f"Tag '{video['tag']}' agregado al video: {video['titulo']}")

Tag 'Tutorial' agregado al video: 📌serigrafia con TINTA PUFF de alto relieve🔥parte 2 #viral #estampados #serigrafiatextil #estampado
Tag 'Tutorial' agregado al video: Aplicando Técnica de puff alto relieve con plastisol #puf #plastisol #serigrafia
Tag 'Historia' agregado al video: Todas las legiones de Marines espaciales explicadas! | Warhammer 40k historia (Parte 1/3)
Tag 'Cultura' agregado al video: Introducción a Warhammer 40.000
Tag 'Hobby' agregado al video: Cómo hacer una base de diorama oceánica realista | Construyendo el AFV Club U-boot Tipo VII D
Tag 'Pop' agregado al video: Starboy x I Feel It Coming
Tag 'Pop' agregado al video: Sunflower, Post Malone, Swae Lee, (Extended Versión)
Tag 'Cine' agregado al video: El Cadav3r de la Novia : La Historia en 1 Video
Tag 'CienciaFicción' agregado al video: EL DIA QUE LA TIERRA EXPLOTO | RESUMEN EN 22 MINUTOS
Tag 'Drama' agregado al video: LAS INJUSTICIAS A UN PROFESOR : PELICULA PERUANA
Tag 'Informativo' agregado al video: América Noti

In [57]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(data_helper, f, ensure_ascii=False, indent=4)
print(f"✅ Etiquetas agregadas y guardadas en {OUTPUT_FILE}")

✅ Etiquetas agregadas y guardadas en sources/videos_with_tags.json


In [ ]:
with open(TAGS_FILE, "r", encoding="utf-8") as f:


In [12]:
for fecha_obj in data:
    fecha = fecha_obj["fecha"]
    videos = fecha_obj["videos"]

    nuevos_videos = [v for v in videos if "tag" not in v]

    if nuevos_videos:
        titulos = "\n".join(v["titulo"] for v in nuevos_videos)
        util_tags = ",".join(new_tags)   

        prompt = f"Asigna una etiqueta (tag) temática de una sola palabra para cada uno de los siguientes títulos de video:\n\n{titulos}"

        if util_tags:
            prompt += f"\n\nPuedes utiliza las siguientes etiquetas o agregar otras nuevas: {util_tags}"
        prompt += "\n\nFormato de respuesta:\n- [Título]=># [Tag]"

        print(f"📝 Generando tags para {len(nuevos_videos)} videos...")
        
        intentos = 0
        while intentos < MAX_RETRIES:
            try:
                response = requests.post(
                    API_URL,
                    headers={
                        "Authorization": f"Bearer {API_KEY}",
                        "Content-Type": "application/json",
                    },
                    json={
                        "model": MODEL,
                        "temperature": 1,
                        "top_p": 1,
                        "messages": [
                            {
                                "role": "system",
                                "content": "Eres un modelo que asigna etiquetas temáticas a títulos de videos musicales.",
                            },
                            {
                                "role": "user",
                                "content": prompt,
                            },
                        ],
                    },
                )

                response.raise_for_status()  # Levanta excepción si código != 200
                content = response.json()["choices"][0]["message"]["content"]
                for line in content.split("\n"):
                    if "=># " in line:
                        try:
                            titulo, tag = line.strip("-").split("=># ", 1)
                            tag_clean = tag.strip().lower()

                            # Comprobar si no está ya en la lista (convertimos todos los elementos a minúsculas para comparar)
                            if tag_clean not in [t.lower() for t in new_tags]:
                                new_tags.append(tag.strip())
                            
                            tag_store[titulo.strip()] = tag.strip()
                        except ValueError:
                            continue

                # Guardar tags actualizados
                with open(TAGS_FILE, "w", encoding="utf-8") as f:
                    json.dump(tag_store, f, indent=2, ensure_ascii=False)

                

                # Recorrer cada objeto en la lista
                for item in data_helper:
                    for video in item["videos"]:
                        titulo = video.get("titulo")  # O la clave real del título
                        if titulo in tag_store:
                                video["tag"] = tag_store[titulo]
                                
                with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
                    json.dump(data_helper, f, indent=2, ensure_ascii=False)
                
                break  # Éxito, salir del while

            except requests.exceptions.HTTPError as e:
                if response.status_code == 429:
                    intentos += 1
                    print(f"⚠️ Rate limit alcanzado. Esperando {WAIT_SECONDS} segundos antes de reintentar (intento {intentos}/{MAX_RETRIES})...")
                    time.sleep(WAIT_SECONDS)
                else:
                    print(f"❌ Error HTTP: {e}")
                    break  # No intentamos más con este error

            except Exception as e:
                print(f"❌ Error inesperado: {e}")
                break

        else:
            print("❌ Se agotaron los reintentos tras error 429. Saltando este prompt.")
            continue

        # Aquí procesas content normalmente y continúas con tu código

        # (Opcional) espera corta para espaciar llamadas
        time.sleep(1)

📝 Generando tags para 23 videos...
📝 Generando tags para 1 videos...
📝 Generando tags para 1 videos...
📝 Generando tags para 13 videos...
📝 Generando tags para 1 videos...
📝 Generando tags para 1 videos...


In [ ]:
with open(TAGS_FILE, "r", encoding="utf-8") as f:
    tags = json.load(f)

In [30]:
with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
    last_try = json.load(f)

In [32]:
for item in last_try:
    for video in item["videos"]:
        if not video.get("tag"):
            print(f"❌ No se encontró tag para el video: {video['titulo']}")

❌ No se encontró tag para el video: 📌serigrafia con TINTA PUFF de alto relieve🔥parte 2 #viral #estampados #serigrafiatextil #estampado
❌ No se encontró tag para el video: HALO-100 (240*320MM)  FORTEVER brand Halo-100 embroidery machine
❌ No se encontró tag para el video: ☑️Configurá #ESLint con #Airbnb y #Prettier.


In [ ]:
tag_store['The Peak']

In [ ]:
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# Recorrer cada objeto en la lista
for item in data:
    if "videos" in item:
        for video in item["videos"]:
            titulo = video.get("titulo")  # O la clave real del título
            if titulo in tag_store:
                video["tag"] = tag_store[titulo]

In [ ]:
data[0]['videos'][0]

In [ ]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"✅ Proceso completado. Resultado guardado en '{OUTPUT_FILE}'")

## META_TAG

In [ ]:
with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [33]:
new_tags = []
for item in data:
    for video in item["videos"]:
        if "tag" in video:
            if video['tag'] not in new_tags:
                new_tags.append(video['tag'])

In [34]:
len(new_tags)

68

In [35]:
new_tags

['Urbano',
 'Debate',
 'Alternativo',
 'Electrónico',
 'Rock',
 'Electro',
 'Lounge',
 'Anime',
 'Pop',
 'Indie',
 'Alternative',
 'Informativo',
 'Comedia',
 'Reflexión',
 'Tutorial',
 'Relajación',
 'Animación',
 'Satírico',
 'Verano',
 'Ciencia',
 'Tecnología',
 'Drama',
 'Sociedad',
 'Motivación',
 'Social',
 'Educación',
 'K-Pop',
 'Latin',
 'Aventura',
 'CienciaFicción',
 'Música',
 'Historia',
 'Cultura',
 'Parodia',
 'Salud',
 'Religión',
 'Hobby',
 'Entretenimiento',
 'Electró',
 'Videojuegos',
 'Acción',
 'Juegos',
 'Rap',
 'Hip-Hop',
 'Unboxing',
 'Viajes',
 'Cine',
 'Jazz',
 'Clásica',
 'Debates',
 'Humor',
 'Dramas',
 'Consejos',
 'Vida',
 'RegionalMexicano',
 'Motociclismo',
 'Dance',
 'Comics',
 'Terror',
 'Fantasía',
 'J-Pop',
 'Documental',
 'Creatividad',
 'Sociedaddebate',
 'Tradicional',
 'Instrumental',
 'Soul',
 'Infomativo']

In [36]:
import requests

def get_best_thumbnail(video_id):
    base_url = f"https://i.ytimg.com/vi/{video_id}/"
    quality_order = [
        "maxresdefault.jpg",
        "sddefault.jpg",
        "hqdefault.jpg",
        "mqdefault.jpg",
        "default.jpg"
    ]
    
    for quality in quality_order:
        url = base_url + quality
        response = requests.head(url)
        if response.status_code == 200:
            return url  # Primera válida encontrada
    return None  # Ninguna funcionó

def extract_video_id(video_url):
    # Extrae el ID del video desde la URL de YouTube
    if "v=" in video_url:
        return video_url.split("v=")[1].split("&")[0]
    return None

# Ejemplo de uso con tu objeto
video_obj = {
    "titulo": "El Episodio de Spiderman que Muestra Que No Todo Depende de Ti",
    "thumbnail": "https://i.ytimg.com/vi/rRGo9-q_Uhw/hqdefault.jpg",
    "videoUrl": "https://www.youtube.com/watch?v=rRGo9-q_Uhw&t=310s",
    "canal": "Mau Lucero",
    "duracion": "13:12",
    "tag": "Acción"
}

video_id = extract_video_id(video_obj["videoUrl"])

if video_id:
    best_thumb = get_best_thumbnail(video_id)
    if best_thumb:
        video_obj["thumbnail"] = best_thumb
        print(f"✅ Miniatura mejorada encontrada: {best_thumb}")
    else:
        print("❌ No se pudo encontrar una miniatura válida. Usando la original.")
else:
    print("❌ No se pudo extraer el ID del video.")


✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/rRGo9-q_Uhw/maxresdefault.jpg


In [38]:
bad_miniature = []

In [40]:
for item in data:
    for video in item["videos"]:
        video_id = extract_video_id(video["videoUrl"])
        if video_id:
            best_thumb = get_best_thumbnail(video_id)
            if best_thumb:
                video["thumbnail"] = best_thumb
                print(f"✅ Miniatura mejorada encontrada: {best_thumb}")
            else:
                bad_miniature.append(video)    
                print("❌ No se pudo encontrar una miniatura válida. Usando la original.")        
        else:
            bad_miniature.append(video)
            print("❌ No se pudo extraer el ID del video.")

✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/pn8RmRET-I8/sddefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/PTLw6GIbRzU/hqdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/RkETqdBxY3c/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/6bBMPgapnwc/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/rpvhRsNKX1I/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/M0WVtz4awOI/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/MgNItWdfEIU/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/WHKlMwYHb34/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/SR89b0qqRAg/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/MQ9K6EbQUxE/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/AmEIemQfk34/maxresdefault.jpg
✅ Miniatura mejorada encontrada: https://i.ytimg.com/vi/j-RpvI

In [47]:
with open(OUTPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [48]:
count = 0
for item in data:
    for video in item["videos"]:
        if not video.get("tag"):
            count += 1

print(f"❌ No se encontró tag para {count} videos")

❌ No se encontró tag para 40 videos


In [42]:
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [45]:
for item in bad_miniature:
    print(f"❌ No se encontró miniatura para el video: {video['thumbnail']}")

❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://i.ytimg.com/vi/jNIb9H2A90k/maxresdefault.jpg
❌ No se encontró miniatura para el video: https://